"""
Module: Run SW RRTM Wrapper (sw_rrtm_wrapper.ipynb)

Created on 2024

@authors: Olivia Salaben and Jianyu Zheng
"""

In [1]:
import numpy as np
from netCDF4 import Dataset

from nbconvert.preprocessors import ExecutePreprocessor
import nbformat

def execute_notebook(notebook_path):
    with open(notebook_path) as f:
        nb = nbformat.read(f, as_version=4)
    
    # Create an instance of the ExecutePreprocessor class
    executor = ExecutePreprocessor(timeout=-1)
    
    # Execute the notebook
    executor.preprocess(nb, {'metadata': {'path': '.'}})
    
    print("Notebook executed successfully.")

# Specify the path to the notebook you want to execute
notebook_path = "/my_run_rrtm_sw_coord.ipynb"

# Call the execute_notebook function with the notebook path
execute_notebook(notebook_path)

In [2]:
#----------
# using Ashok's isolation weighted data from 2006
#----------

sza_Dataset = Dataset('inso_weighted_SZA_seasonal_DustCOMMgrid.nc')
print(sza_Dataset) #[DJF, JJA, MAM, SON,lat,lon]

lat_sza = np.array(sza_Dataset.variables['lat'][:])
lon_sza = np.array(sza_Dataset.variables['lon'][:])
sza_weighted_DJF = np.array(sza_Dataset.variables['isza_DJF'][:])
sza_weighted_JJA = np.array(sza_Dataset.variables['isza_JJA'][:])
sza_weighted_MAM = np.array(sza_Dataset.variables['isza_MAM'][:])
sza_weighted_SON = np.array(sza_Dataset.variables['isza_SON'][:])

sza_weighted_all = [sza_weighted_DJF,sza_weighted_MAM,sza_weighted_JJA, sza_weighted_SON]

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    dimensions(sizes): lon(144), lat(96)
    variables(dimensions): float32 lon(lon), float32 lat(lat), float32 isza_DJF(lat, lon), float32 isza_MAM(lat, lon), float32 isza_JJA(lat, lon), float32 isza_SON(lat, lon)
    groups: 


In [3]:
#Re-order -180 to be +180 and move to end of array and reorder sza data
lon_sza[lon_sza == -180] = 180
lon_sza = np.sort(lon_sza)
print(lon_sza)

sza_weighted_DJF = np.concatenate((sza_weighted_DJF[:,1:],sza_weighted_DJF[:,:1]), axis=1)
sza_weighted_MAM = np.concatenate((sza_weighted_MAM[:,1:],sza_weighted_MAM[:,:1]), axis=1)
sza_weighted_JJA = np.concatenate((sza_weighted_JJA[:,1:],sza_weighted_JJA[:,:1]), axis=1)
sza_weighted_SON = np.concatenate((sza_weighted_SON[:,1:],sza_weighted_SON[:,:1]), axis=1)

sza_weighted_all = [sza_weighted_DJF,sza_weighted_MAM,sza_weighted_JJA, sza_weighted_SON]

[-177.5 -175.  -172.5 -170.  -167.5 -165.  -162.5 -160.  -157.5 -155.
 -152.5 -150.  -147.5 -145.  -142.5 -140.  -137.5 -135.  -132.5 -130.
 -127.5 -125.  -122.5 -120.  -117.5 -115.  -112.5 -110.  -107.5 -105.
 -102.5 -100.   -97.5  -95.   -92.5  -90.   -87.5  -85.   -82.5  -80.
  -77.5  -75.   -72.5  -70.   -67.5  -65.   -62.5  -60.   -57.5  -55.
  -52.5  -50.   -47.5  -45.   -42.5  -40.   -37.5  -35.   -32.5  -30.
  -27.5  -25.   -22.5  -20.   -17.5  -15.   -12.5  -10.    -7.5   -5.
   -2.5    0.     2.5    5.     7.5   10.    12.5   15.    17.5   20.
   22.5   25.    27.5   30.    32.5   35.    37.5   40.    42.5   45.
   47.5   50.    52.5   55.    57.5   60.    62.5   65.    67.5   70.
   72.5   75.    77.5   80.    82.5   85.    87.5   90.    92.5   95.
   97.5  100.   102.5  105.   107.5  110.   112.5  115.   117.5  120.
  122.5  125.   127.5  130.   132.5  135.   137.5  140.   142.5  145.
  147.5  150.   152.5  155.   157.5  160.   162.5  165.   167.5  170.
  172.5  175.   177.

In [4]:
##Create run for all seasons
seasons = [0,1,2,3]
seas_name= ['DJF','MAM','JJA','SON']

In [5]:
# Read in MODIS surface albedo data
MERRA_data = Dataset('MERRA2_400.tavgM_2d_rad_Nx.202301.nc4')
MERRA_data_lat = np.array(MERRA_data.variables['lat'][:])
MERRA_data_lon = np.array(MERRA_data.variables['lon'][:])
new_match_lat_index = np.load('new_match_lat_index.npy')
new_match_lon_index = np.load('new_match_lon_index.npy')

In [6]:
def get_month_albedo(year, month):
    filename = f'MERRA2_400.tavgM_2d_rad_Nx.{year}{month:02}.nc4'
    dataset = Dataset(filename)
    albedo = np.array(dataset.variables['ALBEDO'][0])
    return albedo

albedo_2023 = []

for month in range(1, 13):
    albedo_2023.append(get_month_albedo(2023, month))

albedo_2023 = np.array(albedo_2023)

In [7]:
DJF_merra2_albedo_2023 = np.zeros((len(new_match_lat_index),len(new_match_lon_index)))
MAM_merra2_albedo_2023 = np.zeros((len(new_match_lat_index),len(new_match_lon_index)))
JJA_merra2_albedo_2023 = np.zeros((len(new_match_lat_index),len(new_match_lon_index)))
SON_merra2_albedo_2023 = np.zeros((len(new_match_lat_index),len(new_match_lon_index)))
for ilon in range(len(new_match_lon_index)):
    for ilat in range(len(new_match_lat_index)):
        DJF_merra2_albedo_2023[ilat,ilon] = (albedo_2023[11,new_match_lat_index[ilat],new_match_lon_index[ilon]] + albedo_2023[0,new_match_lat_index[ilat],new_match_lon_index[ilon]] + albedo_2023[1,new_match_lat_index[ilat],new_match_lon_index[ilon]])/3
        MAM_merra2_albedo_2023[ilat,ilon] = (albedo_2023[2,new_match_lat_index[ilat],new_match_lon_index[ilon]] + albedo_2023[3,new_match_lat_index[ilat],new_match_lon_index[ilon]] + albedo_2023[4,new_match_lat_index[ilat],new_match_lon_index[ilon]])/3                
        JJA_merra2_albedo_2023[ilat,ilon] = (albedo_2023[5,new_match_lat_index[ilat],new_match_lon_index[ilon]] + albedo_2023[6,new_match_lat_index[ilat],new_match_lon_index[ilon]] + albedo_2023[7,new_match_lat_index[ilat],new_match_lon_index[ilon]])/3
        SON_merra2_albedo_2023[ilat,ilon] = (albedo_2023[8,new_match_lat_index[ilat],new_match_lon_index[ilon]] + albedo_2023[9,new_match_lat_index[ilat],new_match_lon_index[ilon]] + albedo_2023[10,new_match_lat_index[ilat],new_match_lon_index[ilon]])/3
        
all_merra2_albedo_2023 = [DJF_merra2_albedo_2023, MAM_merra2_albedo_2023, JJA_merra2_albedo_2023, SON_merra2_albedo_2023]

In [ ]:
# Later, calculate bulk seasonal DustCOMM DAOD by season -look at North African dust SW DREE method code
#----------
# using DustCOMM 2021 papers b) https://dustcomm.atmos.ucla.edu/
# specifically https://dustcomm.atmos.ucla.edu/data/K21b/ dataset DustCOMM_source_region_DAOD_seas_bin_abs.nc
# assumes aspherical dust shape, PM20 dust
#----------

daod_Dustcomm_PM20_Dataset = Dataset('DustCOMM_source_region_DAOD_annual_bin_abs.nc')
print(daod_Dustcomm_PM20_Dataset) #[season,source,diameter,lat,lon,bin]

lat_Dustcomm_PM20 = np.array(daod_Dustcomm_PM20_Dataset.variables['lat'][:])
lon_Dustcomm_PM20 = np.array(daod_Dustcomm_PM20_Dataset.variables['lon'][:])
mean_Daod_Dustcomm_20PM = np.array(daod_Dustcomm_PM20_Dataset.variables['Mean'][:])
#dustcomm_seas = np.array(daod_Dustcomm_PM20_Dataset.variables['season'][:])

bin_NAf_dust_loading = mean_loading_Dustcomm_20PM[:,:,:,0] + mean_loading_Dustcomm_20PM[:,:,:,1] + mean_loading_Dustcomm_20PM[:,:,:,2]
#bulk_NAf_dust_loading
bin_Sarahan_laoding = mean_loading_Dustcomm_20PM[:,:,:,0] + mean_loading_Dustcomm_20PM[:,:,:,1]
bin_Sahelian_loading = mean_loading_Dustcomm_20PM[:,:,:,2]
print(total_North_African_loading.shape)

## Load DustCOMM North African bulk dust seasonal DAOD

In [ ]:
#----------
# using DustCOMM 2021 papers b) https://dustcomm.atmos.ucla.edu/
# specifically https://dustcomm.atmos.ucla.edu/data/K21b/ dataset DustCOMM_source_region_DAOD_seas_bin_abs.nc
# assumes aspherical dust shape, PM20 dust
#----------

#bulk DAOD data
daod_Dustcomm_PM20_Dataset = Dataset('../../Dust_Optical_Properties/DustCOMM_source_region_DAOD_seas_PM20_abs.nc')
print(daod_Dustcomm_PM20_Dataset) #[season,source,diameter,lat,lon,bin]

lat_Dustcomm_PM20 = np.array(daod_Dustcomm_PM20_Dataset.variables['lat'][:])
lon_Dustcomm_PM20 = np.array(daod_Dustcomm_PM20_Dataset.variables['lon'][:])
mean_Daod_Dustcomm_20PM = np.array(daod_Dustcomm_PM20_Dataset.variables['Mean'][:])
#dustcomm_seas = np.array(daod_Dustcomm_PM20_Dataset.variables['season'][:])
dustcomm_Sources_PM20 = np.array(daod_Dustcomm_PM20_Dataset.variables['source'][:]) 

#Create seasonal North African DAOD as a function of lat and lon
NAf_Daod_Dustcomm_20PM = mean_Daod_Dustcomm_20PM[:,:,0,:] + mean_Daod_Dustcomm_20PM[:,:,1,:] + mean_Daod_Dustcomm_20PM[:,:,2,:]

DJF_NAf_Daod_Dustcomm_20PM = NAf_Daod_Dustcomm_20PM[:,:,0]
MAM_NAf_Daod_Dustcomm_20PM = NAf_Daod_Dustcomm_20PM[:,:,1]
JJA_NAf_Daod_Dustcomm_20PM = NAf_Daod_Dustcomm_20PM[:,:,2]
SON_NAf_Daod_Dustcomm_20PM = NAf_Daod_Dustcomm_20PM[:,:,3]

## Multiply: (SW DRE/column RRTM DAOD) * DustCOMM North African Dust DAOD